<a href="https://colab.research.google.com/github/DmitryKutsev/hse_compling_homework/blob/master/hw5_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!curl --remote-name \
     -H 'Accept: application/vnd.github.v3.raw' \
     --location https://github.com/mannefedov/compling_nlp_hse_course/blob/master/data/habr_texts.txt.zip?raw=true


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   154    0   154    0     0    660      0 --:--:-- --:--:-- --:--:--   660
100   165  100   165    0     0    383      0 --:--:-- --:--:-- --:--:--  161k
100 17.7M  100 17.7M    0     0  9938k      0  0:00:01  0:00:01 --:--:-- 83.0M


In [0]:
mv 'habr_texts.txt.zip?raw=true' habr_texts.txt.zip

In [0]:
!ls

habr_texts.txt	habr_texts.txt.zip  sample_data


In [0]:
!pip install pymorphy2
!pip install nltk
!pip install pyLDAvis
!pip install gensim


     |████████████████████████████████| 51kB 3.8MB/s 
     |████████████████████████████████| 7.1MB 9.0MB/s 
     |████████████████████████████████| 1.6MB 9.5MB/s 
     |████████████████████████████████| 552kB 41.0MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97711 sha256=d7a1548e5396c6b1c54a0a4367f625b76499195d5f0b29a35732077e916970b8
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
  Created wheel for funcy: filename=funcy-1.14-py2.py3-none-any.whl size=32040 sha256=4157c2158cee68819e780ee700aeae5fda9e6da7843b0ee0583af93905d9e720
  Stored in directory: /root/.cache/pip/wheels/20/5a/d8/1d875df03deae6f178dfdf70238cca33f948ef8a6f5209f2eb
Successfully built pyLDAvis funcy


In [0]:
import gensim
import json
import re
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
import pyLDAvis.gensim
import string
from collections import Counter
import warnings
warnings.filterwarnings("ignore")
import gzip
morph = MorphAnalyzer()

In [0]:
import nltk
nltk.download('stopwords')
import csv

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
import zipfile
corpus = open('habr_texts.txt', 'w')
with zipfile.ZipFile('habr_texts.txt.zip', 'r') as archive:
    archive.extractall('')


In [0]:
stops = set(stopwords.words('russian')) | {'gt',}
add_list = ["англ", "поль", "хабр", "джон", "крис", "плохо", "хорошо", "назад", "пожалуйста", "спасибо" "вычисление", "хабра", "мк", "ми", "ос", "где-нибудь", "ия", "мс", "ый"] #добавил непонятные слова, и слова, которые встречаются в большинстве топиков вообще,
# и еще аббривеатуры 
add_list = [stops.add(i) for i in add_list]
def remove_tags_and_latine(text):
    text = re.sub(r'[A-Za-z]+', '', text) #попробую убрать латиницу и комбинации только из цифр
    text = re.sub(r'[0-9.…•"«№“”±€×]+', '', text) #и комбинации из цифр с плохо отфильтровавшимися знаками
    return re.sub(r'<[^>]+>', '', text)


def opt_normalize(texts, top=None):
    uniq = Counter()
    for text in texts:
        uniq.update(text)
    
    norm_uniq = {word:morph.parse(word)[0].normal_form for word, _ in uniq.most_common(top)}
    
    norm_texts = []
    for text in texts:
        
        norm_words = [norm_uniq.get(word) for word in text]
        norm_words = [word for word in norm_words if word and word not in stops]
        norm_texts.append(norm_words)
        
    return norm_texts

def tokenize(text):
    words = [word.strip(string.punctuation) for word in text.split()]
    words = [word for word in words if word]
    
    return words

In [0]:
texts = open('habr_texts.txt').read().splitlines()
texts = opt_normalize([tokenize(remove_tags_and_latine(text.lower())) for text in texts], 40000)

In [0]:
# для нграммов
ph = gensim.models.Phrases(texts, scoring='npmi', threshold=0.4) # threshold можно подбирать
p = gensim.models.phrases.Phraser(ph)
ngrammed_texts = p[texts]

In [0]:
dictionary = gensim.corpora.Dictionary(ngrammed_texts)

In [0]:
dictionary.filter_extremes(no_above=0.1, no_below=20)
dictionary.compactify()

In [0]:
print(dictionary)

Dictionary(6449 unique tokens: ['аргумент', 'базироваться', 'бизнес-логика', 'блог', 'выбираться']...)


In [0]:
corpus = [dictinary.doc2bow(text) for text in texts]

In [0]:
tfidf = gensim.models.TfidfModel(corpus, id2word=dictionary)
corpus = tfidf[corpus]

In [0]:
#lda = gensim.models.LdaMulticore(corpus, 100, id2word=dictinary, eval_every=0)# если поддерживается многопоточность
lda = gensim.models.LdaModel(corpus, 100, alpha="symmetric", id2word=dictinary, eval_every=0, passes=30)
 
# lda = gensim.models.LdaModel(200, id2word=dictinary, passes=5)

## Начну сразу с лучших тем модели LDA:
**Из получившихся мне больше всего понравилась модель с дефолтным параметром alpha и 15 проходами по тексту. При увеличении или уменьшении alpha , и в комбинации с "asymmetric" и "auto", с разными количествами проходов по тексту, становилось, на мой взгляд, хуже.
Вот модель и темы: *** 

lda = gensim.models.LdaModel(corpus, 100, alpha="symmetric", id2word=dictinary, eval_every=0, passes=15)
Когерентность и перплексия тоже у нее получились лучше остальных:
 -7.9273474073139765 перплексия
 0.5202449758837351 когерентность

**И темы, мне показались они хорошими, вроде особо ничего лишнего:**

 (76,
  '0.105*"обучение" + 0.041*"машинный" + 0.041*"искусственный" + 0.039*"интеллект" + 0.030*"нейросеть" + 0.024*"нейронный" + 0.024*"университет" + 0.023*"распознавание" + 0.018*"робот" + 0.017*"компьютерный"'),
 (92,
  '0.171*"игрок" + 0.064*"игровой" + 0.026*"удача" + 0.021*"автомат" + 0.015*"шанс" + 0.014*"дизайнер" + 0.013*"жанр" + 0.012*"случайный" + 0.012*"геймплей" + 0.012*"стратегия"'),
 (34,
  '0.031*"текстура" + 0.025*"моделирование" + 0.023*"расчёт" + 0.022*"линия" + 0.018*"проектирование" + 0.016*"геометрический" + 0.015*"сетка" + 0.015*"компонента" + 0.014*"вершина" + 0.013*"шейдёр"')


**На всякий случай я оставил часть тем, прошу прощения заранее, если не надо было этого делать(можно промотать).** 

  

In [0]:
lda.print_topics()
#lda = gensim.models.LdaModel(corpus, 100, alpha="symmetric", id2word=dictinary, eval_every=0, passes=30)
# перплексия -13.577861529810319 
#  когерентность nan
# 1) тема из стандартной модели с 30 проходами по тексту:
# (26,'0.069*"нейросеть" + 0.059*"обучение" + 0.054*"нейронный" + 0.040*"нейрон" + 0.026*"распознавание" + 0.022*"машинный" + 0.022*"слой" + 0.016*"обучать" + 0.016*"обучаться" + 0.012*"признак"'),

# 2) (114, '0.068*"ракета" + 0.045*"наса" + 0.042*"авария" + 0.037*"космический" + 0.036*"миссия" + 0.029*"станция" + 0.026*"разрушение" + 0.025*"груз" + 0.025*"мкс" + 0.025*"космос"')

#3) (51, '0.083*"протокол" + 0.067*"трафик" + 0.052*"шлюз" + 0.050*"маршрут" + 0.038*"маршрутизатор" + 0.028*"маска" + 0.028*"сетевой" + 0.027*"соединение" + 0.026*"сессия" + 0.022*"сегмент"'),

 (83,
  '0.062*"датчик" + 0.037*"температура" + 0.024*"воздух" + 0.023*"вода" + 0.021*"прибор" + 0.020*"площадь" + 0.019*"комната" + 0.015*"умный" + 0.015*"шарик" + 0.013*"измерение"'),
 (92,
  '0.171*"игрок" + 0.064*"игровой" + 0.026*"удача" + 0.021*"автомат" + 0.015*"шанс" + 0.014*"дизайнер" + 0.013*"жанр" + 0.012*"случайный" + 0.012*"геймплей" + 0.012*"стратегия"'),
 (34,

[(99,
  '0.069*"вм" + 0.053*"жалоба" + 0.037*"штраф" + 0.022*"январь" + 0.021*"игорь" + 0.019*"федеральный" + 0.019*"российский" + 0.016*"углубиться" + 0.012*"поступить" + 0.011*"напомнить"'),
 (94,
  '0.053*"зашифровать" + 0.035*"временной" + 0.034*"шифрование" + 0.023*"пароль" + 0.021*"раскрываться" + 0.020*"бизнес-логика" + 0.020*"метаданные" + 0.020*"уничтожение" + 0.020*"дублировать" + 0.019*"аннотация"'),
 (44,
  '0.027*"корпус" + 0.026*"камера" + 0.021*"мм" + 0.021*"аккумулятор" + 0.019*"дисплей" + 0.017*"гб" + 0.016*"светодиод" + 0.016*"ноутбук" + 0.015*"датчик" + 0.015*"разъём"'),
 (71,
  '0.068*"регулирование" + 0.036*"мкс" + 0.031*"смс" + 0.018*"поправка" + 0.011*"пощупать" + 0.000*"пройти_успешно" + 0.000*"пятнадцать" + 0.000*"прозрачный" + 0.000*"разум" + 0.000*"рождество"'),
 (67,
  '0.122*"клетка" + 0.087*"ген" + 0.036*"днк" + 0.033*"бактерия" + 0.022*"образование" + 0.021*"молекула" + 0.021*"геном" + 0.018*"белок" + 0.014*"белка" + 0.012*"ребёнок"'),
 (47,
  '0.076*"при

In [0]:
#lda = gensim.models.LdaModel(corpus, 200, alpha="symmetric", id2word=dictinary, eval_every=0, passes=15) + tfidf
lda.print_topics() 
# -16.851228932060963 перплексия
# 0.37793586846030697 когерентность

[(136,
  '0.081*"блокировка" + 0.056*"чиновник" + 0.055*"заблокировать" + 0.047*"мотивация" + 0.043*"провайдер" + 0.027*"посещать" + 0.025*"печально" + 0.024*"куб" + 0.023*"где-нибудь" + 0.023*"анонимность"'),
 (112,
  '0.134*"присоединение" + 0.019*"нож" + 0.011*"индивидуально" + 0.000*"хирург" + 0.000*"сегмент" + 0.000*"техподдержка" + 0.000*"продлить" + 0.000*"миниатюрный" + 0.000*"гбит/с" + 0.000*"изобретение"'),
 (46,
  '0.042*"заносить" + 0.037*"телевизор" + 0.034*"лестница" + 0.030*"студенческий" + 0.025*"вакуум" + 0.024*"склад" + 0.023*"зимний" + 0.020*"недовольный" + 0.020*"отставать" + 0.017*"лечь"'),
 (20,
  '0.072*"анимировать" + 0.025*"скромный" + 0.023*"классно" + 0.020*"затруднить" + 0.000*"анонсировать" + 0.000*"математика" + 0.000*"анонс" + 0.000*"сертификация" + 0.000*"грубый" + 0.000*"ознакомление"'),
 (114,
  '0.068*"ракета" + 0.045*"наса" + 0.042*"авария" + 0.037*"космический" + 0.036*"миссия" + 0.029*"станция" + 0.026*"разрушение" + 0.025*"груз" + 0.025*"мкс" + 0.

In [0]:
lda.print_topics() 
#lda = gensim.models.LdaModel(corpus, 200, alpha="symmetric", id2word=dictinary, eval_every=0, passes=20)

# -7.989605661044144 перплексия
# 0.5061547540553559когерентность

[(111,
  '0.176*"электроника" + 0.067*"школьник" + 0.046*"физика" + 0.040*"транзистор" + 0.032*"школьный" + 0.032*"логический" + 0.026*"цифровой" + 0.025*"калькулятор" + 0.021*"электронный" + 0.020*"разбор"'),
 (125,
  '0.134*"уязвимость" + 0.039*"диспетчер" + 0.033*"вредоносный" + 0.027*"веб-сервер" + 0.023*"произвольный" + 0.020*"уязвимый" + 0.018*"веб-приложение" + 0.015*"вирус" + 0.015*"исследователь" + 0.013*"процедура"'),
 (68,
  '0.041*"угол" + 0.030*"расстояние" + 0.027*"попадание" + 0.019*"квадрат" + 0.018*"верхний" + 0.016*"вектор" + 0.016*"плоскость" + 0.014*"луч" + 0.014*"нижний" + 0.013*"расчёт"'),
 (95,
  '0.197*"клетка" + 0.043*"деление" + 0.042*"ген" + 0.035*"образование" + 0.034*"днк" + 0.034*"молекула" + 0.029*"вещество" + 0.020*"белок" + 0.018*"концентрация" + 0.017*"белка"'),
 (179,
  '0.297*"домен" + 0.087*"восстановление" + 0.039*"переадресация" + 0.030*"локальный" + 0.030*"политика" + 0.024*"мониторинг" + 0.024*"обратный" + 0.024*"вручную" + 0.023*"доменный" + 0.

In [0]:
lda.print_topics() 
#lda = gensim.models.LdaModel(corpus, 100, alpha="symmetric", id2word=dictinary, eval_every=0, passes=15)
#
# -7.9273474073139765 перплексия
# 0.5202449758837351 когерентность

[(32,
  '0.094*"реальность" + 0.076*"лаборатория" + 0.074*"исследователь" + 0.060*"форум" + 0.052*"проведение" + 0.051*"респондент" + 0.041*"эксперт" + 0.041*"трансляция" + 0.034*"опрос" + 0.031*"наблюдать"'),
 (75,
  '0.062*"стикер" + 0.034*"аккаунт" + 0.028*"блокировка" + 0.025*"динамический" + 0.022*"короткий" + 0.017*"блокировать" + 0.017*"удалять" + 0.014*"длинный" + 0.012*"добавление" + 0.012*"удаление"'),
 (51,
  '0.083*"протокол" + 0.067*"трафик" + 0.052*"шлюз" + 0.050*"маршрут" + 0.038*"маршрутизатор" + 0.028*"маска" + 0.028*"сетевой" + 0.027*"соединение" + 0.026*"сессия" + 0.022*"сегмент"'),
 (30,
  '0.110*"двигатель" + 0.032*"пуск" + 0.031*"ступень" + 0.027*"ракета" + 0.024*"ракета-носитель" + 0.024*"авария" + 0.018*"спутник" + 0.017*"испытание" + 0.014*"орбита" + 0.014*"корабль"'),
 (11,
  '0.112*"китай" + 0.073*"китайский" + 0.047*"правительство" + 0.043*"провайдер" + 0.025*"заблокировать" + 0.023*"контент" + 0.022*"обход" + 0.017*"блокировка" + 0.016*"международный" + 0.0

In [0]:
lda.print_topics() 
#lda = gensim.models.LdaModel(corpus, 100, alpha="auto", id2word=dictinary, eval_every=0, passes=5)
# -8.066113052577508 перплексия
# 0.49396613933049643 когерентность
#  Попробуем добавить биграммы.

[(10,
  '0.260*"дата-центр" + 0.110*"доверие" + 0.081*"регистрация" + 0.064*"образец" + 0.064*"фактор" + 0.060*"дц" + 0.030*"катастрофа" + 0.024*"снимать" + 0.016*"чинить" + 0.016*"риск"'),
 (3,
  '0.108*"сообщество" + 0.091*"формула" + 0.087*"гонка" + 0.050*"телеметрия" + 0.048*"сектор" + 0.032*"трасса" + 0.031*"консорциум" + 0.029*"скотт" + 0.026*"секция" + 0.022*"заполнить"'),
 (52,
  '0.293*"двигатель" + 0.068*"ступень" + 0.054*"пуск" + 0.040*"содержание" + 0.031*"ракета-носитель" + 0.021*"преобразователь" + 0.020*"игорь" + 0.019*"контур" + 0.018*"надёжность" + 0.017*"ток"'),
 (18,
  '0.116*"доход" + 0.068*"налог" + 0.060*"работник" + 0.052*"налоговый" + 0.043*"предприниматель" + 0.040*"ип" + 0.032*"взнос" + 0.028*"деятельность" + 0.024*"расход" + 0.021*"грузовик"'),
 (87,
  '0.286*"тк" + 0.070*"дорога" + 0.069*"сегмент" + 0.058*"идентификатор" + 0.048*"трекинг" + 0.038*"агрегат" + 0.031*"кубик" + 0.029*"интервал" + 0.029*"трек" + 0.019*"уникальность"'),
 (36,
  '0.411*"энергия" + 

In [0]:
lda.print_topics() 
#lda = gensim.models.LdaModel(corpus, 100, alpha="auto", id2word=dictinary, eval_every=0, passes=5)
# -8.066113052577508 перплексия
# 0.49396613933049643 когерентность
# Пока это лучшая модель в ЛДА. Попробуем добавить ниграммы. Судя по всему, лучше не становится

[(24,
  '0.125*"переключатель" + 0.076*"интуиция" + 0.068*"пропорция" + 0.065*"умение" + 0.057*"продвижение" + 0.051*"гарантия" + 0.050*"портфолио" + 0.041*"золото" + 0.031*"пирамида" + 0.030*"по-новому"'),
 (51,
  '0.222*"приёмник" + 0.096*"стрелка" + 0.058*"ван" + 0.052*"конструкция" + 0.052*"полёт" + 0.044*"регулятор" + 0.034*"оборот" + 0.028*"верификация" + 0.028*"высота" + 0.026*"м/с"'),
 (56,
  '0.249*"лог" + 0.194*"тк" + 0.071*"ибо" + 0.063*"логово" + 0.048*"сообщить" + 0.042*"автопилот" + 0.032*"дтп" + 0.029*"экспорт" + 0.026*"экспортировать" + 0.020*"записать"'),
 (54,
  '0.139*"болезнь" + 0.080*"‘’" + 0.065*"германия" + 0.065*"транзистор" + 0.049*"больной" + 0.043*"болеть" + 0.026*"век" + 0.020*"диагноз" + 0.019*"закон" + 0.019*"лечение"'),
 (40,
  '0.178*"частица" + 0.123*"квантовый" + 0.095*"чёрный" + 0.067*"секция" + 0.040*"горизонт" + 0.037*"где-нибудь" + 0.036*"акция" + 0.025*"дыра" + 0.022*"материальный" + 0.015*"физика"'),
 (6,
  '0.264*"музыка" + 0.086*"музыкальный" +

## **NMF**

In [0]:
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd

In [0]:
stexts = [' '.join(text) for text in texts]

In [0]:
vectorizer = TfidfVectorizer(max_features=1000, min_df=10, max_df=0.5, ngram_range=(1,2))
X = vectorizer.fit_transform(stexts)

In [0]:
# n_components - главный параметр в NMF, это количество тем. 
# Если данных много, то увеличения этого параметра сильно увеличивает время обучения
model = NMF(n_components=60, alpha=1.0)

In [0]:
model.fit(X)

NMF(alpha=1.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
    n_components=60, random_state=None, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)

In [0]:
model.components_.shape # матрица темы на слова

(60, 1000)

In [0]:
model.transform(X).shape # матрица документы на темы

(4121, 60)

In [0]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [0]:
feat_names = vectorizer.get_feature_names()

**# Здесь я менял параметры и расширил список топиков, чтобы посмотреть ,что стало хуже, потому что сразу все отработало сравнительно неплохо сразу. От увеличения трэшхолда энграммов лучше не стало(но и сильно хуже тоже), до единицы alpha улучшает результат в метрике, после единицы становится хуже. Визуально неплохо везде. Мне кажется гораздо лучше, чем LDA.**
Опять же, оставил несколько промежуточных результатов.

# **Модель взял по метрике:**
NMF(n_components=60, alpha=1.0), TfidfVectorizer(max_features=1000, min_df=10, max_df=0.5, ngram_range=(1,2)), 50.742120893393576

**Топ тем:**

**1) атака  безопасность  защита  информационный  пароль  доступ  инфраструктура  информация  интернет**

**2) проект  разработчик  разработка  создание  сборка  продукт  инструмент  идея  заказчик**

**3) модель  обучение  алгоритм  сеть  признак  слой  набор  вектор  област**ь

**Темы NMF мне нравятся больше, чем в LDA, и к тому же, с ней не пришлось столько возиться.** 


In [0]:
#NMF(n_components=60, alpha=1.0), TfidfVectorizer(max_features=1000, min_df=10, max_df=0.5, ngram_range=(1,2)), 50.742120893393576
top_words = model.components_.argsort()[:,:-10:-1]
for i in range(top_words.shape[0]):
    words = [feat_names[j] for j in top_words[i]]
    print(i, "  ".join(words))

0 какой  какой то  что  что то  дело  ваш  проблема  хотеть  знать
1 класс  метод  тип  реализация  вызов  объект  библиотека  свойство  интерфейс
2 игра  игрок  игровой  играть  разработчик  уровень  разработка  мир  мобильный
3 сервис  облачный  контейнер  доступ  пользователь  облако  запрос  приложение  платформа
4 устройство  умный  мобильный  производитель  управление  драйвер  смартфон  передача  обновление
5 приложение  мобильный  разработка  разработчик  ваш  платформа  библиотека  веб  создание
6 компания  сотрудник  рынок  миллион  бизнес  директор  продажа  крупный  миллиард
7 пользователь  продукт  пароль  информация  доступ  пользовательский  аудитория  запись  интерфейс
8 учёный  исследование  научный  исследователь  университет  результат  температура  группа  обнаружить
9 файл  папка  скрипт  строка  пароль  создать  настройка  имя  запускать
10 код  библиотека  ошибка  исходный код  исходный  написать  компилятор  переменный  разработчик
11 сайт  реклама  домен  конте

In [0]:
#model = NMF(n_components=40, alpha=1.0), TfidfVectorizer(max_features=1000, min_df=10, max_df=0.5, ngram_range=(1,3)), 53.07881347524317
top_words = model.components_.argsort()[:,:-10:-1]
for i in range(top_words.shape[0]):
    words = [feat_names[j] for j in top_words[i]]
    print(i, "  ".join(words))

0 человек  какой  что  какой то  что то  ваш  дело  хотеть  знать
1 класс  метод  тип  объект  реализация  свойство  код  вызов  библиотека
2 игра  игрок  игровой  играть  разработчик  уровень  разработка  мир  мобильный
3 сервис  облачный  технология  облако  платформа  инфраструктура  услуга  рынок  мобильный
4 смартфон  телефон  экран  модель  гб  корпус  цена  кнопка  ноутбук
5 приложение  мобильный  разработка  разработчик  платформа  ваш  пользователь  библиотека  веб
6 компания  сотрудник  рынок  бизнес  продукт  директор  миллион  продажа  крупный
7 пользователь  сообщение  продукт  пароль  информация  браузер  сервис  страница  приложение
8 человек  учёный  мозг  исследование  пациент  исследователь  научный  университет  жизнь
9 файл  папка  скрипт  строка  настройка  пароль  создать  программа  добавить
10 код  библиотека  ошибка  исходный код  исходный  написать  программа  разработчик  компилятор
11 сайт  страница  реклама  браузер  контент  домен  веб  адрес  ссылка
12 се

In [0]:
#model = NMF(n_components=40, alpha=2.0), TfidfVectorizer(max_features=1000, min_df=10, max_df=0.5, ngram_range=(1,3)), 53.92069351629626
top_words = model.components_.argsort()[:,:-10:-1]
for i in range(top_words.shape[0]):
    words = [feat_names[j] for j in top_words[i]]
    print(i, "  ".join(words))

0 человек  какой  что  какой то  что то  ваш  дело  хотеть  знать
1 код  библиотека  ошибка  метод  функция  написать  исходный код  исходный  компилятор
2 игра  игрок  игровой  играть  разработчик  уровень  мир  разработка  мобильный
3 сервер  домен  настройка  адрес  база  конфигурация  клиент  запрос  скрипт
4 устройство  смартфон  мобильный  телефон  производитель  умный  датчик  технология  режим
5 приложение  мобильный  разработка  разработчик  ваш  платформа  пользователь  библиотека  версия
6 компания  сотрудник  рынок  бизнес  миллион  российский  директор  крупный  миллиард
7 пользователь  продукт  пароль  браузер  информация  приложение  доступ  интерфейс  проблема
8 человек  учёный  мозг  исследование  пациент  исследователь  научный  университет  жизнь
9 файл  скрипт  папка  строка  команда  настройка  пакет  модуль  добавить
10 программа  компьютер  компьютерный  программирование  программный  программист  программный обеспечение  университет  обеспечение
11 сайт  страниц

In [0]:
#model = NMF(n_components=40, alpha=1.0), TfidfVectorizer(max_features=1000, min_df=10, max_df=0.5, ngram_range=(1,2)), 52.811976340327874
top_words = model.components_.argsort()[:,:-10:-1]
for i in range(top_words.shape[0]):
    words = [feat_names[j] for j in top_words[i]]
    print(i, "  ".join(words))

0 человек  какой  что  какой то  что то  ваш  дело  хотеть  знать
1 класс  метод  тип  объект  реализация  свойство  вызов  библиотека  интерфейс
2 игра  игрок  игровой  играть  разработчик  уровень  мир  мобильный  разработка
3 сервер  клиент  домен  адрес  настройка  конфигурация  база  нагрузка  выделить
4 устройство  смартфон  мобильный  телефон  производитель  умный  технология  датчик  режим
5 приложение  мобильный  разработка  разработчик  платформа  пользователь  ваш  библиотека  веб
6 компания  сотрудник  рынок  бизнес  миллион  российский  директор  крупный  продукт
7 пользователь  сервис  клиент  сообщение  письмо  продукт  информация  интерфейс  ваш
8 человек  учёный  мозг  исследование  пациент  исследователь  научный  университет  группа
9 файл  папка  скрипт  строка  пароль  создать  настройка  имя  программа
10 код  библиотека  ошибка  исходный код  исходный  написать  программа  компилятор  разработчик
11 сайт  страница  реклама  браузер  контент  домен  веб  пользоват

In [0]:
#model = NMF(n_components=40), TfidfVectorizer(max_features=1000, min_df=10, max_df=0.3, ngram_range=(1,2))
top_words = model.components_.argsort()[:,:-5:-1]
for i in range(top_words.shape[0]):
    words = [feat_names[j] for j in top_words[i]]
    print(i, "  ".join(words))

0 какой то  что то  думать  вообще
1 российский  рынок  рубль  миллион
2 игра  игрок  игровой  играть
3 класс  метод  библиотека  вызов
4 устройство  мобильный  умный  датчик
5 продукт  мобильный  платформа  инструмент
6 учёный  мозг  клетка  исследование
7 язык  программирование  программист  программа
8 атака  безопасность  защита  программа
9 файл  папка  скрипт  строка
10 доклад  конференция  тема  участник
11 сайт  страница  браузер  реклама
12 тест  тестирование  тестовый  библиотека
13 запрос  таблица  строка  ответ
14 сервер  настройка  домен  конфигурация
15 космический  спутник  аппарат  земля
16 обучение  изображение  алгоритм  модель
17 сеть  трафик  связь  интернет
18 бот  скрипт  канал  друг
19 печать  принтер  мм  материал
20 дата центр  центр  дата  оборудование
21 виртуальный  машина  виртуальный машина  резервный
22 автомобиль  машина  технология  метр
23 вселенная  энергия  теория  свет
24 сигнал  звук  частота  диапазон
25 клиент  бизнес  услуга  сотрудник
26 книга 

In [0]:
model.reconstruction_err_

50.742120893393576